<a href="https://colab.research.google.com/github/esserilee/Innopolis/blob/main/_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

предобработка текста

In [ ]:
#я решила выбрать создание чат-бота, который будет генерировать ответы на 3 популярных интернет вопроса
#выбрала датасет с набором вопросов и ответов, собранных с платформы Яндекс

import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# русские стоп-слова
russian_stopwords = set(stopwords.words('russian'))

# загружаем датасет
url = "https://huggingface.co/datasets/its5Q/yandex-q/resolve/main/answers.jsonl.gz"
df = pd.read_json(url, lines=True, compression="gzip")
from datasets import Dataset

# преобразуем в HF Dataset
dataset = Dataset.from_pandas(df_small[['question', 'answer']])

# очистка текста нижний регистр
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].astype(str).str.lower()
        df[col] = df[col].apply(lambda x: re.sub(r'[^а-яё\s]', '', x))

# токенизация
for col in df.columns:
    if df[col].dtype == 'object':
        df[col + '_processed'] = df[col].apply(
            lambda text: [word for word in text.split() if word not in russian_stopwords]
        )

print(df.head())

from datasets import Dataset

# преобразуем df
df_small = df.head(50)  # первые 50 строк, тк колаб выдавал ошибку по переполнению памяти
dataset = Dataset.from_pandas(df_small[['question', 'answer']])





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                  description  \
0                               
1                               
2  сколько стоят хорошие окна   
3                               
4                               

                                            question  \
0  как войти в роблокс если не работает верификац...   
1  под чьей властью находились восточные города в...   
2                         сколько стоят хорошие окна   
3               можно ли заниматься любовью на пасху   
4  почему у ноутбука очень часто включается венти...   

                                              answer  \
0  никак но можно попробовать найти сайты которые...   
1  восточные города находятся под властью людей к...   
2  на цену лучше не ориентироваться это тот случа...   
3  конечно можно пост прошел теперь можно есть ск...   
4  если у вас открыто  вкладок это особо не нагру...   

             description_processed  \
0                               []   
1                               []   
2  [сколько, 

Обучение модели

In [ ]:
#загружаем модель
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token #Использование EOS токена как PAD токена
model.config.pad_token_id = tokenizer.eos_token_id

def preprocess_function(examples):
    # создаем входной текст
    inputs = [f"<s>Вопрос: {q}</s>" for q in examples["question"]]
    targets = [f"<s>Ответ: {a}</s>" for a in examples["answer"]]

    # токенизация входа
    model_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors=None
    )

    # токенизация
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors=None
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess_function, batched=True)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


проверка результатов генерации ответа

In [ ]:
#три популярных запроса по версии гугл 1)13 ноября праздник? 2)сухой закон 3)переводы сбп
def chat(prompt, model, max_new_tokens=20, temperature=0.8):
    input_text = f"Вопрос: {prompt} Ответ:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if "Ответ:" in response:
        response = response.split("Ответ:")[-1].strip()
    return response



for q in [
    "13 ноября праздник?",
    "сухой закон",
    "переводы сбп"
]:
    print(f"👤: {q}")
    print(f"🤖: {chat(q, model)}\n")

👤: 13 ноября праздник?
🤖: 13 ноября праздник.



26977016	svetka-u

👤: сухой закон
🤖: Закон сохранения энергии.  Применительно к энергии это понятие означает, что энергия, проходя через тело

👤: переводы сбп
🤖: Здравствуйте!  Возможно ли перевести на русский язык произведения "Мойдодыр" и "Приз



Отвечат более менее по теме, но мы все равно сделаем дообучение

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#указываем аргументы дообучения модели
import os
os.environ["WANDB_DISABLED"] = 'true'

training_args = TrainingArguments(
    output_dir="./rugpt3_chatbot", # Папка для сохранения всех артефактов обучения
    per_device_train_batch_size = 2, # Сколько записей приходится на один батч
    num_train_epochs = 3, # Количество эпох
    learning_rate = 5e-5, # Скорость обучения
    fp16 = torch.cuda.is_available(), # На каком устройстве происходит обучение
    logging_steps = 10, # Через сколько шагов выдается информация об обучении модели
    save_total_limit = 1, # Сколько раз логируется процесс обучения модели
    report_to = "none" # Отключает интеграцию с WANDB, которая с сентября заблокирована в России
)

#дообучение
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("content/my_rugpt_chatbot")
tokenizer.save_pretrained("content/my_rugpt_chatbot")

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "content/my_rugpt_chatbot"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

for q in [
    "13 ноября праздник?",
    "сухой закон",
    "переводы сбп"
]:
    print(f"👤: {q}")
    print(f"🤖: {chat(q, model)}\n")

/tmp/ipython-input-2903588486.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,7.391800
20,4.885600
30,3.964100
40,3.184600
50,3.042700
60,2.467400
70,2.359600


👤: 13 ноября праздник?
🤖: нет такого праздника, кроме как день рождения ребенка и день рождения ребенка в этот день не отмечался никогда

👤: сухой закон
🤖: нет, не законно. как правильно написать заявление в полицию о правонарушении в  случае если не

👤: переводы сбп
🤖: а для чего нужны лицензии на работу в гос.службе?
Для того что бы получать доход от



После дообучения ответы все равно попадают в тему (по слову закон, например, модель выдает что-то, но ответы все равно бредовые) колаб не позволяет использовать весь датасет и сложную модель, тк переполняется память, но по концепции можно увидеть, что это работает